In [13]:
import pandas as pd
import operator as op
class FASTA_tools(object):
    def __init__(self,file):
        self.file = file
        f=open(self.file)
        fasta = f.read()
        record = pd.DataFrame
        fasta_id = fasta.split('>')
        del fasta_id[0]
        Sequence_identifier = []
        Sequence_description = []
        Sequence = []
        Seq_len = [] 
        for sequence in fasta_id:
            lines = sequence.splitlines() #separate each line, first line(item) is the ID&description
            Sequence_identifier.append(lines[0].split(',')[0]) #separate by ',' first item is ID
            Sequence_description.append(lines[0].split(',')[1]) #second item is description
            Sequence.append( ''.join(lines[1:-1])) #join the rest items in the list as the sequence
            Seq_len.append(len(''.join(lines[1:-1])))
        record['Seq_id'] = Sequence_identifier
        record['Seq_description'] = Sequence_description
        record['Sequence'] = Sequence
        record['Sequence_length'] = Seq_len
        def checkORF(seq, f): #giving the sequence and readingframe, return all ORF as dict with keys-sequence,value-start position,
                          # of ORF, length of the longest ORF
            ORF = {}
            stop_codon = ['TAA', 'TAG','TGA']
            for i in range(f,len(seq)-3,3):
                if seq[i:i+3].upper() == 'ATG':
                    for j in range(i+3,len(seq)-3,3):
                        if seq[j:j+3].upper() in stop_codon:
                            ORF[seq[i:j+3]] = i
                            break
            return ORF, len(ORF) #len(max(ORF, key = len))
       

        def totalORF(seq): #total reading frame for all three possibility
            total_number = checkORF(seq,1)[1]+checkORF(seq,2)[1]+checkORF(seq,3)[1]
            all_ORF = dict(list(checkORF(seq,1)[0].items())+list(checkORF(seq,2)[0].items())+list(checkORF(seq,3)[0].items()))
            return all_ORF,total_number

        def longestORF(seq): #return the longest ORF,start position,the length
            frame = max((totalORF(seq)[0]),key = len)
            length = len(frame)
            position = totalORF(seq)[0][frame]
            return frame,position,length

        def longORF(seq,f):
            if checkORF(seq,f)[0]!={}:
                frame = max(checkORF(seq,f)[0])
                length = len(frame)
                position = checkORF(seq,f)[0][frame]
            else:
                frame = ''
                length = 0
                position = 0
            return frame, position, length
        record['total_ORF'] = record['Sequence'].apply(totalORF) 
        record['total_ORF'] = record['total_ORF'].apply(op.itemgetter(1)) #total reading frame calculated
        record['longest_ORF'] = record['Sequence'].apply(longestORF).apply(op.itemgetter(0))
        record['longest_ORF_position'] = record['Sequence'].apply(longestORF).apply(op.itemgetter(1))
        record['longest_ORF_length'] = record['longest_ORF'].apply(len)
        record['total_ORF1'] = record.apply(lambda x:checkORF(x['Sequence'],1),axis =1).apply(op.itemgetter(1))
        record['longest_ORF1'] = record.apply(lambda x:longORF(x['Sequence'],1),axis = 1).apply(op.itemgetter(0))
        record['longest_ORF1_length'] = record['longest_ORF1'].apply(len)
        record['longest_ORF1_position'] = record.apply(lambda x:longORF(x['Sequence'],1),axis = 1).apply(op.itemgetter(1))
        record['total_ORF2'] = record.apply(lambda x:checkORF(x['Sequence'],2),axis =1).apply(op.itemgetter(1))
        record['longest_ORF2'] = record.apply(lambda x:longORF(x['Sequence'],2),axis = 1).apply(op.itemgetter(0))
        record['longest_ORF2_length'] = record['longest_ORF2'].apply(len)
        record['longest_ORF2_position'] = record.apply(lambda x:longORF(x['Sequence'],2),axis = 1).apply(op.itemgetter(1))
        record['total_ORF3'] = record.apply(lambda x:checkORF(x['Sequence'],3),axis =1).apply(op.itemgetter(1))
        record['longest_ORF3'] = record.apply(lambda x:longORF(x['Sequence'],3),axis = 1).apply(op.itemgetter(0))
        record['longest_ORF3_length'] = record['longest_ORF3'].apply(len)
        record['longest_ORF3_position'] = record.apply(lambda x:longORF(x['Sequence'],3),axis = 1).apply(op.itemgetter(1))
        self.record = record
    def getRecord(self):
        return self.record
    def count_seq(self):
        return len(record)
    def check_length(self):
        longest = len(max(record['Sequence'],key=len))
        longest_id = record[record['Sequence']==max(record['Sequence'],key=len)]['Seq_id']
        shortest = len(min(record['Sequence'],key = len))
        shortest_id = record[record['Sequence']==min(record['Sequence'],key=len)]['Seq_id']
        return longest_id,longest,shortest_id,shortest
    
    '''this function check length of each record and the according header 
        of the record'''
    def findrepeats(n):
        segments = {}
        seg = []
        record = data.getRecord()
        for item in record['Sequence']:
            for i in range(0,len(item)-n):
                if item[i:i+n] not in seg:
                    seg.append(item[i:i+n])
                    segments[item[i:i+n]] = 0
                else:
                    segments[item[i:i+n]]+=1
        highest_freq = max(segments.values())
        highest_freq_repeats = []
        if highest_freq > 0:       
            for i,j in segments.items():
                if j == highest_freq:
                    highest_freq_repeats.append(i)
        else:
            highest_freq_repeats.append('No Repeats Identified')
            
        return highest_freq, highest_freq_repeats
